## Autonomous Planner Agent Implementation

In [1]:
import json
from dotenv import load_dotenv
import logging
from openai import OpenAI
import chromadb

from agents.scanner_agent import ScannerAgent

In [2]:
load_dotenv(override=True)
openai = OpenAI() 
MODEL = "gpt-5-mini"

## Start with some test data and test agentic function/tool calling

In [3]:
test_results = ScannerAgent().test_scan()
test_results

DealSelection(deals=[Deal(product_description="The Hisense R6 Series 55R6030N is a 55-inch 4K UHD Roku Smart TV that offers stunning picture quality with 3840x2160 resolution. It features Dolby Vision HDR and HDR10 compatibility, ensuring a vibrant and dynamic viewing experience. The TV runs on Roku's operating system, allowing easy access to streaming services and voice control compatibility with Google Assistant and Alexa. With three HDMI ports available, connecting multiple devices is simple and efficient.", price=178.0, url='https://www.dealnews.com/products/Hisense/Hisense-R6-Series-55-R6030-N-55-4-K-UHD-Roku-Smart-TV/484824.html?iref=rss-c142'), Deal(product_description='The Poly Studio P21 is a 21.5-inch LED personal meeting display designed specifically for remote work and video conferencing. With a native resolution of 1080p, it provides crystal-clear video quality, featuring a privacy shutter and stereo speakers. This display includes a 1080p webcam with manual pan, tilt, and

In [4]:
### Mockup/Simulation function
def scan_the_internet_for_bargains() -> str: 
    """ This tool scans the internet for great deals and gets a curated list of promising deals """
    print("Fake function to scan the internet -- this returns a hardcoded set of deals.")
    return test_results.model_dump_json()

In [5]:
### Mockup/Simulation function 
def estimate_true_value(description: str) -> str:
    """
    This tool estimates the true value of a product based on a text description of it
    """
    print(f"Fake function to estimating true value of {description[:20]}... - this always returns $300") 
    return f"Product {description} has an estimated true value of $300"

In [6]:
def notify_user_of_deal(description: str, deal_price: float, estimated_true_value: float, url: str) -> str:
    """
    This tool notifies the user of a great deal, given a description of it, the price of the deal, and the estimated true value
    """
    print(f"Fake function to notify user of {description} which costs {deal_price} and estimate is {estimated_true_value}")
    return "notification has been sent"

In [7]:
notify_user_of_deal("a new iphone", 100, 1000, "https://www.apple.com/iphone")

Fake function to notify user of a new iphone which costs 100 and estimate is 1000


'notification has been sent'

## Schema Definition 
Build Json schema structure to describe tools to the LLM model.

In [8]:
scan_function = {
        "name": "scan_the_internet_for_bargains",
        "description": "Returns top bargains scraped from the internet along with the price each item is being offered for",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": [],
            "additionalProperties": False
        }
    }

estimate_function = {
    "name": "estimate_true_value",
    "description": "Given the description of an item, estimate how much it is actually worth",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item to be estimated"
            },
        },
        "required": ["description"],
        "additionalProperties": False
    }
}

notify_function = {
    "name": "notify_user_of_deal",
    "description": "Send the user a push notification about the single most compelling deal; only call this one time",
    "parameters": {
        "type": "object",
        "properties": {
            "description": {
                "type": "string",
                "description": "The description of the item itself scraped from the internet"
            },
            "deal_price": {
                "type": "number",
                "description": "The price offered by this deal scraped from the internet"
            }
            ,
            "estimated_true_value": {
                "type": "number",
                "description": "The estimated actual value that this is worth"
            }
            ,
            "url": {
                "type": "string",
                "description": "The URL of this deal as scraped from the internet"
            }
        },
        "required": ["description", "deal_price", "estimated_true_value", "url"],
        "additionalProperties": False
    }
}

In [9]:
tools = [
    {"type": "function", "function": scan_function}, 
    {"type": "function", "function": estimate_function}, 
    {"type": "function", "function": notify_function}
]

In [10]:
tools

[{'type': 'function',
  'function': {'name': 'scan_the_internet_for_bargains',
   'description': 'Returns top bargains scraped from the internet along with the price each item is being offered for',
   'parameters': {'type': 'object',
    'properties': {},
    'required': [],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'estimate_true_value',
   'description': 'Given the description of an item, estimate how much it is actually worth',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string',
      'description': 'The description of the item to be estimated'}},
    'required': ['description'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'notify_user_of_deal',
   'description': 'Send the user a push notification about the single most compelling deal; only call this one time',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string',
      'description

In [11]:
system_message = "You find great deals on bargain products using your tools, and notify the user of the best bargain."
user_message = """
First, use your tool to scan the internet for bargain deals. Then for each deal, use your tool to estimate its true value.
Then pick the single most compelling deal where the price is much lower than the estimated true value, and use your tool to notify the user.
Then just reply OK to indicate success.
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

In [12]:
messages

[{'role': 'system',
  'content': 'You find great deals on bargain products using your tools, and notify the user of the best bargain.'},
 {'role': 'user',
  'content': '\nFirst, use your tool to scan the internet for bargain deals. Then for each deal, use your tool to estimate its true value.\nThen pick the single most compelling deal where the price is much lower than the estimated true value, and use your tool to notify the user.\nThen just reply OK to indicate success.\n'}]

In [13]:
def handle_tool_call(message):
    """
    Call the tools associated with this message
    """
    results = []

    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        ### access any corresponding function dynamically (with no explicit func name matching)
        tool = globals().get(tool_name) 
        # print("TOOL: ", tool)
        ### result is what the corresponding/chosen function returns
        result = tool(**arguments) if tool else {}
        results.append(
            {"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id}
        )

    # print("RESULTS: ", results)

    return results

In [14]:
### Append the tool call response into the messages list so LLM continues to process it
while True:
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print("RESPONSE: ", response)
    base_path = response.choices[0]
    if base_path.finish_reason == "tool_calls": 
        message = base_path.message
        results = handle_tool_call(message)
        messages.append(message) # assistant's tool call message with the tool_calls property
        messages.extend(results) # tool response
        continue
    else: 
        break

response.choices[0].message.content

RESPONSE:  ChatCompletion(id='chatcmpl-D055vNB4iCqh5MU2ln1EFHBpnyc6b', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_1IavQHiEcx9KKsh1kYmnP0cZ', function=Function(arguments='{}', name='scan_the_internet_for_bargains'), type='function')]))], created=1768911955, model='gpt-5-mini-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=217, prompt_tokens=357, total_tokens=574, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=192, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Fake function to scan the internet -- this returns a hardcoded set of deals.
RESPONSE:  ChatCompletion(id='chatcmpl-D0560

'OK'

## Autonomous Planning Agent

Switching to the real functions.

In [3]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [4]:
DB = "products_vectorstore"
client = chromadb.PersistentClient(path=DB)
collection = client.get_or_create_collection("products")

In [5]:
from agents.autonomous_planning_agent import AutonomousPlanningAgent
agent = AutonomousPlanningAgent(collection=collection)

/Volumes/VTG/Dev/C_5/Projects/week8/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is initializing...
INFO:root:[Scanner Agent] Scanner Agent is initializing...
INFO:root:[Scanner Agent] Scanner Agent is set!
INFO:root:[Ensemble Agent] Initializing Ensemble Agent...
INFO:root:[Specialist Agent] Specialist Agent is initializing -- connecting to Modal...
INFO:root:[Specialist Agent] Specialist Agent is ready!
INFO:root:[Frontier Agent] Initializing Frontier Agent...
INFO:root:[Frontier Agent] Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:ro

In [6]:
agent.plan()

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is kicking off a run
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is calling Scanner Agent to scan deals online
INFO:root:[Scanner Agent] Scanner Agent is fetching available deals...


TOOL CALL MSG:  ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_7GUUfZnfnBIGGDRHlpRLxoze', function=Function(arguments='{}', name='scan_the_internet_for_bargains'), type='function')])



ARG TYPE:  <class 'dict'>



ARGS:  {}





INFO:root:[Scanner Agent] Scanner Agent received 30 new fresh deals!
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI client...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is calling Ensemble Agent to estimate the true value
INFO:root:[Ensemble Agent] Running Ensemble Agent - collaborating with specialist, frontier and neural network agents...
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model... If this is a cold start, this might take some time to wake it up


TOOL CALL MSG:  ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_V398oo3uOgoSPJ34nDQgced6', function=Function(arguments='{"description": "Refurbished EcoFlow Delta 2 Max is a high-capacity portable power station featuring a 2048Wh LiFePO4 battery chemistry for long cycle life and thermal stability. The unit provides multiple AC outlets (six 20A), an LCD display with LED indicators, USB ports including USB-C, a 12V car adapter, Wi‑Fi connectivity, and multiple charging modes including AC and solar input, with 2400W continuous output and 4800W surge capacity. It\'s designed for portable backup power, camping, and off-grid use, with a robust set of outputs for appliances and electronics."}', name='estimate_true_value'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_PyURRLJp2A7fUhYulquyF4Iy', function=Function(arguments='{"description": "Humsienk 

INFO:root:[Specialist Agent] Specialist Agent completed - predicting $999.00
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
Batches: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.19it/s]
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5-mini with context including 5 similar products...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $1499.00
INFO:root:[Neural Network Agent] Neural Network Agent is processing the price estimation...
INFO:root:[Neural Network Agent] Neural Network Agent completed -- predicting $127.57
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $1449.00
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is calling Ensemble Agent to estimat

...

ARG TYPE:  <class 'dict'>



ARGS:  {'description': 'Ningmei Ryzen 7 gaming desktop is a prebuilt PC featuring an AMD Ryzen 7 5700G 3.8GHz 8-core CPU, 16GB of RAM, a 1TB SSD, and a Radeon RX 6600/LE 8GB GPU. It ships with Windows 11 Home and is configured for mainstream to midrange gaming and content creation, offering a balanced combination of CPU multi-threaded performance and a dedicated GPU for 1080p gaming and productivity workflows.'}





INFO:root:[Specialist Agent] Specialist Agent completed - predicting $950.00
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products
Batches: 100%|███████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.86it/s]
INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5-mini with context including 5 similar products...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $799.00
INFO:root:[Neural Network Agent] Neural Network Agent is processing the price estimation...
INFO:root:[Neural Network Agent] Neural Network Agent completed -- predicting $670.13
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $814.10
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[

TOOL CALL MSG:  ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_VeXVPnycJVrQHdmh7FHqpFWj', function=Function(arguments='{"description":"Humsienk 12V 314Ah Bluetooth LiFePO4 battery is a deep-cycle lithium iron phosphate battery offering very high cycle life (15,000+ cycles) and integrated Bluetooth monitoring for real-time status and diagnostics. It includes a 200A battery management system (BMS) for protection and reliable high-current operation, making it suitable for solar energy storage, RVs, marine use, and off-grid systems where long lifespan and smart monitoring are priorities.","deal_price":288.0,"estimated_true_value":1804.1,"url":"https://www.dealnews.com/Humsienk-12-V-314-Ah-Bluetooth-Li-Fe-PO4-Battery-for-288-free-shipping/21804046.html?iref=rss-c142"}', name='notify_user_of_deal'), type='function')])



ARG TYPE:  <class 'dict'>



ARGS:  {'descrip

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Messaging Agent] Messaging Agent is sending the notification message...
INFO:root:[Messaging Agent] Messaging Agent has completed!
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent completed with: OK


Opportunity(deal=Deal(product_description='Humsienk 12V 314Ah Bluetooth LiFePO4 battery is a deep-cycle lithium iron phosphate battery offering very high cycle life (15,000+ cycles) and integrated Bluetooth monitoring for real-time status and diagnostics. It includes a 200A battery management system (BMS) for protection and reliable high-current operation, making it suitable for solar energy storage, RVs, marine use, and off-grid systems where long lifespan and smart monitoring are priorities.', price=288.0, url='https://www.dealnews.com/Humsienk-12-V-314-Ah-Bluetooth-Li-Fe-PO4-Battery-for-288-free-shipping/21804046.html?iref=rss-c142'), estimate=1804.1, discount=1516.1)